# Mine developers' Profiles, Gender, and Location
<!-- Sharif Ahmed -->

In [ ]:
import pandas as pd
from github import Github
import json

## Mine GitHub User Profiles

In [ ]:
prs = pd.read_pickle('../data/mined/all/cass_prs.pkl')
prcs = pd.read_pickle('../data/mined/all/cass_pr_comments.pkl')
iss = pd.read_pickle('../data/mined/all/cass_issues.pkl')
iscs = pd.read_pickle('../data/mined/all/cass_issue_comments.pkl')

In [ ]:
users = []
for df in [prs, prcs, iss,iscs]:
    users += df.user_login.tolist())

In [ ]:
ulist = list(set(users))

In [ ]:
from github import Auth, Github
GHTOKEN = 'XXX'
auth = Auth.Token(GHTOKEN)
g = Github(auth=auth)

def get_gh_profile(login):
    user = g.get_user(login)
    t = dict()
    t["login"]=user.login
    t["id"]=user.id
    t["node_id"]=user.node_id
    t["type"]=user.type
    t["site_admin"]=user.site_admin
    t["name"]=user.name
    t["company"]=user.company
    t["blog"]=user.blog
    t["location"]=user.location
    t["email"]=user.email
    t["hireable"]=user.hireable
    t["bio"]=user.bio
    t["twitter_username"]=user.twitter_username
    t["followers"]=user.followers
    t["following"]=user.following
    t["created_at"]=user.created_at
    t["updated_at"]=user.updated_at
    return t


In [ ]:
udicts = []
for u in ulist:
 udicts.append(get_gh_profile(u))

In [ ]:
profiles = pd.DataFrame(udicts)

In [ ]:
profiles.head()

In [ ]:
profiles.to_csv('../data/profiles.csv')

In [ ]:
profiles =pd.read_csv('../data/profiles.csv',index_col=0)

## Gender

In [ ]:
import sys
import os

# Determine the path to the subscript folder
n_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'models', 'gender_guesser'))
if n_dir not in sys.path:
    sys.path.append(n_dir)

In [ ]:
import detector as gender
d = gender.Detector(case_sensitive=False)

In [ ]:
profiles['gender'] = profiles.name.apply(lambda n: '' if type(n)!= str else d.get_gender(n.split()[0]))


In [ ]:
profiles['gender'].value_counts()

gender
male             4941
                 3709
unknown          1947
andy              453
female            287
mostly_male       286
mostly_female      58
Name: count, dtype: int64

In [ ]:
profiles['gender']=profiles['gender'].apply(lambda n: n if type(n)!= str else n.split('_')[-1])

In [ ]:
profiles['gender'].value_counts()

gender
male       5227
           3709
unknown    1947
andy        453
female      345
Name: count, dtype: int64

In [ ]:
profiles.loc[profiles['gender'] =='', 'gender']='unknown'

In [ ]:
profiles['gender'].value_counts()

gender
unknown    5656
male       5227
andy        453
female      345
Name: count, dtype: int64

## Location

In [ ]:
locdf = pd.DataFrame(profiles['location'].unique().tolist()[1:],columns=['ul'])


In [ ]:
locdf

,ul
0,"San Francisco, CA, USA"
1,Champaign
2,"Aitken Basin, Luna"
3,"Dublin, Ireland"
4,"New York, NY"
...,...
1865,"Maynooth, Ireland"
1866,"Lund, Sweden"
1867,"Taiwan, Taipei"
1868,"Canberra, ACT, Australia"


In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Safari")
def get_loc(loc):
    location = geolocator.geocode(loc)
    return location.address

In [ ]:
get_loc("175 5th Avenue NYC").split(',')[-1].strip()

'United States'

In [ ]:
locdf['address']=locdf['ul'].apply(get_loc)

In [ ]:
locdf.address

0                San Francisco, California, United States
1       Champaign, Champaign County, Illinois, United ...
2                                                     NaN
3                Dublin, County Dublin, Leinster, Ireland
4                                 New York, United States
                              ...                        
1865          Maynooth, County Kildare, Leinster, Ireland
1866        Lund, Lund Municipality, Skåne County, Sweden
1867                                       Taipei, Taiwan
1868    Canberra, District of Canberra Central, Austra...
1869                           Shijiazhuang, Hebei, China
Name: address, Length: 1870, dtype: object

In [ ]:

locdf['country'] = locdf.address.apply(lambda c: c.split(',')[-1].strip() if type(c) ==str else '' )
locdf['country']

0       United States
1       United States
2                    
3             Ireland
4       United States
            ...      
1865          Ireland
1866           Sweden
1867           Taiwan
1868        Australia
1869            China
Name: address, Length: 1870, dtype: object

In [ ]:
locdf

,ul,address,country
0,"San Francisco, CA, USA","San Francisco, California, United States",United States
1,Champaign,"Champaign, Champaign County, Illinois, United ...",United States
2,"Aitken Basin, Luna",NaN,NaN
3,"Dublin, Ireland","Dublin, County Dublin, Leinster, Ireland",Ireland
4,"New York, NY","New York, United States",United States
...,...,...,...
1865,"Maynooth, Ireland","Maynooth, County Kildare, Leinster, Ireland",Ireland
1866,"Lund, Sweden","Lund, Lund Municipality, Skåne County, Sweden",Sweden
1867,"Taiwan, Taipei","Taipei, Taiwan",Taiwan
1868,"Canberra, ACT, Australia","Canberra, District of Canberra Central, Austra...",Australia


### Correction

In [ ]:
locdf[locdf.ul=='UK']#-> Central African Republic

,ul,address,country
260,UK,"Ouaka, Central African Republic",Central African Republic


In [ ]:
locdf.loc[locdf.ul=='UK','country']='United Kingdom'

In [ ]:
locdf[locdf.ul=='UK'].country

260    United Kingdom
Name: country, dtype: object

### Profile ⇠ Gender & Location

In [ ]:
profiles = pd.merge(profiles, locdf[['ul','country']], left_on='location', right_on='ul', how='left',  suffixes=('_prof', '_geoloc'))

In [ ]:
profiles.head()

In [ ]:
profiles.country.isna().sum()

np.int64(7035)

In [ ]:
profiles.gender.value_counts(1)

gender
unknown    0.484205
male       0.447479
andy       0.038781
female     0.029535
Name: proportion, dtype: float64

In [ ]:
profiles.country.value_counts()

country
United States              1603
Germany                     480
China                       418
United Kingdom              324
France                      209
                           ... 
Palestinian Territories       1
Nepal                         1
Panama                        1
UTC−08:00 standard time       1
Kazakhstan                    1
Name: count, Length: 90, dtype: int64

In [ ]:

profiles.to_csv('../data/profiles_loc.csv', index=None)

Thank you!